# 나만의 RAG 챗봇 구현

In [159]:
from dotenv import load_dotenv
load_dotenv()

True

In [160]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader('./data/foreigner.pdf')
documents = loader.load()

# 개행문자 제거
for doc in documents:
    doc.page_content = doc.page_content.replace('\n', ' ')

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # 각 chunk의 최대 문자 수 (기본값: 1000)
    chunk_overlap=100,    # 인접한 텍스트 조각 간 겹치는 문자 수 (기본값: 200)
    separators=["\n\n", "\n", ".", " ", ""]  # 명시적 구분자 설정
)                       # seperators: 텍스트 분할 구분자 우선순위 (기본값: ['\n\n', '\n', ' ', ''])

docs = splitter.split_documents(documents)
for doc in docs:
    print(doc.page_content)


법제처                                                            1                                                       국가법령정보센터 외국인근로자의 고용 등에 관한 법률   외국인근로자의 고용 등에 관한 법률 ( 약칭: 외국인고용법 ) [시행 2022. 12. 11.] [법률 제18929호, 2022. 6. 10., 일부개정] 고용노동부 (외국인력담당관) 044-202-7151        제1장 총칙 <개정 2009. 10. 9.>   제1조(목적) 이 법은 외국인근로자를 체계적으로 도입ㆍ관리함으로써 원활한 인력수급 및 국민경제의 균형 있는 발전 을 도모함을 목적으로 한다. [전문개정 2009. 10. 9
. [전문개정 2009. 10. 9.]   제2조(외국인근로자의 정의) 이 법에서 “외국인근로자”란 대한민국의 국적을 가지지 아니한 사람으로서 국내에 소재하 고 있는 사업 또는 사업장에서 임금을 목적으로 근로를 제공하고 있거나 제공하려는 사람을 말한다. 다만, 「출입국 관리법」 제18조제1항에 따라 취업활동을 할 수 있는 체류자격을 받은 외국인 중 취업분야 또는 체류기간 등을 고려 하여 대통령령으로 정하는 사람은 제외한다. [전문개정 2009. 10. 9.]   제3조(적용 범위 등) ① 이 법은 외국인근로자 및 외국인근로자를 고용하고 있거나 고용하려는 사업 또는 사업장에 적 용한다. 다만, 「선원법」의 적용을 받는 선박에 승무(乘務)하는 선원 중 대한민국 국적을 가지지 아니한 선원 및 그 선 원을 고용하고 있거나 고용하려는 선박의 소유자에 대하여는 적용하지 아니한다
. ② 외국인근로자의 입국ㆍ체류 및 출국 등에 관하여 이 법에서 규정하지 아니한 사항은 「출입국관리법」에서 정하 는 바에 따른다. [전문개정 2009. 10. 9.]   제4조(외국인력정책위원회) ① 외국인근로자의 고용관리 및 보호에 관한 주요 사항을 심의ㆍ의결하기 위하여 국무총리 소속으로 외국인력정

In [161]:
# 임베딩 모델 생성
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma.vectorstores import Chroma

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')
vector_store = Chroma.from_documents(docs, embedding_model)


In [162]:
retriever = vector_store.as_retriever(
    search_type = 'mmr',
    search_kwargs={
        'k': 5,          # 더 많은 문서 검색 (3 → 5)
        'fetch_k': 20,   # 다양성을 위해 더 많은 후보 검색
        'lambda_mult': 0.7  # 관련성과 다양성 간의 균형 설정        
    }
)

In [163]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ('system', '''당신은 외국인 노동자들의 법적 권리와 보호에 관한 전문 챗봇입니다. 
한국어가 모국어가 아닌 사람처럼 어눌하게 말하되, 핵심 법률 정보는 정확하게 전달합니다.

주어진 context를 참고하여 답변하세요. context에 관련 정보가 없다면, 
"그런 정보는 없어요. 하지만 비슷한 상황에서는 이렇게 할 수 있어요"라고 시작하는 일반적인 조언을 제공하세요.

특히 다음 주제에 관한 질문에 초점을 맞추세요:
- 임금체불 (월급, 돈 관련 문제)
- 폭력, 폭행, 학대 문제
- 부당한 처우
- 근로계약 위반
- 사업장 변경 절차
- 체류자격 문제

답변할 때는 위의 안내에 따라 간결하고 명확하게, 외국인 노동자가 이해하기 쉽게 설명하세요.'''),
    ('user', '''질문: {query}

참고 context:
{context}''')
])

In [164]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.5
)

In [165]:
from langchain_core.output_parsers import StrOutputParser

# context 생성
query = input()
retrievers = retriever.batch([query])
context_text = "\n".join([doc.page_content for doc in retrievers[0]])

chain = prompt | model | StrOutputParser()

KeyboardInterrupt: Interrupted by user

In [ ]:
result = chain.invoke({'query': query, 'context': context_text})

In [ ]:
print(result)

그런 정보는 없어요. 하지만 비슷한 상황에서는 이렇게 할 수 있어요. 

만약 집이 없고 갈 곳이 없다면, 먼저 회사에 말해보세요. 회사는 외국인 노동자에게 기숙사를 제공해야 해요. 기숙사가 없다면, 다른 숙소를 찾거나, 도움을 요청할 수 있는 기관이나 단체에 연락해보세요. 

또한, 지역의 외국인 지원 센터나 상담기관에 가서 도움을 받을 수 있어요. 그곳에서 주거 문제에 대한 조언을 받을 수 있습니다.
